In [7]:
import pandas as pd

url = "https://www.basketball-reference.com/draft/"

# read ALL draft years in one call
tables = pd.read_html(url)

df = tables[0]  # first table = full draft history

# inspect columns
print(df.columns)

# save immediately
df.to_csv("nba_draft_all_years.csv", index=False)

df.head(), df.tail()


Index(['Draft', 'Lg', 'Team', 'Player', 'College'], dtype='object')


(   Draft   Lg               Team              Player         College
 0   2025  NBA   Dallas Mavericks        Cooper Flagg            Duke
 1   2024  NBA      Atlanta Hawks  Zaccharie Risacher             NaN
 2   2023  NBA  San Antonio Spurs   Victor Wembanyama             NaN
 3   2022  NBA      Orlando Magic      Paolo Banchero            Duke
 4   2021  NBA    Detroit Pistons     Cade Cunningham  Oklahoma State,
     Draft   Lg                     Team           Player  \
 74   1951  NBA        Baltimore Bullets  Gene Melchiorre   
 75   1950  NBA           Boston Celtics      Chuck Share   
 76   1949  BAA  Providence Steamrollers    Howie Shannon   
 77   1948  BAA  Providence Steamrollers   Andy Tonkovich   
 78   1947  BAA       Pittsburgh Ironmen  Clifton McNeely   
 
                       College  
 74                    Bradley  
 75              Bowling Green  
 76               Kansas State  
 77                   Marshall  
 78  Texas Wesleyan University  )

In [9]:
import os
import time
import logging
from typing import Dict, Optional

import pandas as pd

# Try to use cloudscraper if available (helps with Cloudflare)
try:
    import cloudscraper
    scraper = cloudscraper.create_scraper(
        browser={"browser": "chrome", "platform": "windows", "mobile": False}
    )
    USE_CLOUDSCRAPER = True
except ImportError:
    import requests
    scraper = requests.Session()
    USE_CLOUDSCRAPER = False

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

BASE_URL = "https://www.basketball-reference.com/draft/NBA_{}.html"
START_YEAR = 2000
END_YEAR = 2025

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0 Safari/537.36"
    ),
    "Accept-Language": "en-US,en;q=0.9",
}

OUT_DIR = "draft_tables"
os.makedirs(OUT_DIR, exist_ok=True)


def fetch_html(year: int) -> Optional[str]:
    """Fetch raw HTML for one draft year."""
    url = BASE_URL.format(year)
    logging.info(f"Fetching {url}  (cloudscraper={USE_CLOUDSCRAPER})")

    try:
        resp = scraper.get(url, headers=HEADERS, timeout=20)
    except Exception as e:
        logging.error(f"{year}: request failed: {e}")
        return None

    if resp.status_code != 200:
        logging.warning(f"{year}: HTTP {resp.status_code}, skipping.")
        return None

    return resp.text


def parse_draft_table(html: str, year: int) -> Optional[pd.DataFrame]:
    """Parse and lightly clean the main draft table for a single year."""
    try:
        tables = pd.read_html(html)
    except ValueError as e:
        logging.error(f"{year}: pandas.read_html failed: {e}")
        return None

    if not tables:
        logging.warning(f"{year}: no tables found")
        return None

    # On these pages, the first table is the full draft
    df = tables[0].copy()

    # Drop completely empty columns (from multi-level headers etc.)
    df = df.dropna(axis=1, how="all")

    # Remove repeated header rows that appear inside the body
    for col in ("Pk", "Rnd"):
        if col in df.columns:
            df = df[df[col] != col]

    # Keep only rows that look like real picks (Pk not NaN and numeric)
    if "Pk" in df.columns:
        df = df[~df["Pk"].isna()]

        def is_int_like(x):
            try:
                int(str(x))
                return True
            except Exception:
                return False

        df = df[df["Pk"].apply(is_int_like)]
        df["Pk"] = df["Pk"].astype(int)

    df["Year"] = year
    df = df.reset_index(drop=True)
    return df


def scrape_each_draft_separately(
    start_year: int = START_YEAR,
    end_year: int = END_YEAR,
) -> Dict[int, pd.DataFrame]:
    """
    Scrape each draft table from start_year to end_year (inclusive).

    Returns:
        dict mapping {year: DataFrame} and saves one CSV per year.
    """
    draft_dfs: Dict[int, pd.DataFrame] = {}

    for year in range(start_year, end_year + 1):
        html = fetch_html(year)
        if html is None:
            continue

        df_year = parse_draft_table(html, year)
        if df_year is None or df_year.empty:
            logging.warning(f"{year}: no usable rows, not saving.")
            continue

        # Save individual CSV
        out_path = os.path.join(OUT_DIR, f"nba_draft_{year}.csv")
        df_year.to_csv(out_path, index=False)
        logging.info(f"{year}: saved {len(df_year)} rows to {out_path}")

        # Store in dict for in-memory use
        draft_dfs[year] = df_year

        # Be polite with a small delay
        time.sleep(1.0)

    return draft_dfs


# ---- run it ----
draft_dfs = scrape_each_draft_separately()

# Quick sanity check: show years successfully scraped
print("Years scraped:", sorted(draft_dfs.keys()))
for y in sorted(draft_dfs.keys())[:3]:   # first 3 years as a preview
    print(f"\n=== {y} draft, first 5 rows ===")
    print(draft_dfs[y].head())



/tmp/ipython-input-3380566444.py:63: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)
/tmp/ipython-input-3380566444.py:63: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)
/tmp/ipython-input-3380566444.py:63: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)
/tmp/ipython-input-3380566444.py:63: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)
/tmp/ipython-input-3380566444.py:63: FutureWarning: Pass

Years scraped: [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]

=== 2000 draft, first 5 rows ===
  Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0         Round 1  \
                  Rk                 Pk                 Tm          Player   
0                  1                  1                NJN   Kenyon Martin   
1                  2                  2                VAN  Stromile Swift   
2                  3                  3                LAC    Darius Miles   
3                  4                  4                CHI    Marcus Fizer   
4                  5                  5                ORL     Mike Miller   

              Unnamed: 5_level_0 Totals                      ... Shooting  \
      College                Yrs      G     MP    PTS   TRB  ...      FT%   
0  Cincinnati                 15    757  23134   9325  5159  ...     .629   
1         LSU   

In [10]:
import pandas as pd
import os

folder = "draft_tables"

all_dfs = []

for year in range(2000, 2025 + 1):
    filename = f"nba_draft_{year}.csv"
    path = os.path.join(folder, filename)

    df = pd.read_csv(path)

    # Add Year column (if not already present)
    df["Year"] = year

    all_dfs.append(df)

len(all_dfs)


26

In [12]:
for year, df in zip(range(2000, 2026), all_dfs):
    print(year, df.columns.tolist())


2000 ['Unnamed: 0_level_0', 'Unnamed: 1_level_0', 'Unnamed: 2_level_0', 'Round 1', 'Round 1.1', 'Unnamed: 5_level_0', 'Totals', 'Totals.1', 'Totals.2', 'Totals.3', 'Totals.4', 'Shooting', 'Shooting.1', 'Shooting.2', 'Per Game', 'Per Game.1', 'Per Game.2', 'Per Game.3', 'Advanced', 'Advanced.1', 'Advanced.2', 'Advanced.3', 'Year']
2001 ['Unnamed: 0_level_0', 'Unnamed: 1_level_0', 'Unnamed: 2_level_0', 'Round 1', 'Round 1.1', 'Unnamed: 5_level_0', 'Totals', 'Totals.1', 'Totals.2', 'Totals.3', 'Totals.4', 'Shooting', 'Shooting.1', 'Shooting.2', 'Per Game', 'Per Game.1', 'Per Game.2', 'Per Game.3', 'Advanced', 'Advanced.1', 'Advanced.2', 'Advanced.3', 'Year']
2002 ['Unnamed: 0_level_0', 'Unnamed: 1_level_0', 'Unnamed: 2_level_0', 'Round 1', 'Round 1.1', 'Unnamed: 5_level_0', 'Totals', 'Totals.1', 'Totals.2', 'Totals.3', 'Totals.4', 'Shooting', 'Shooting.1', 'Shooting.2', 'Per Game', 'Per Game.1', 'Per Game.2', 'Per Game.3', 'Advanced', 'Advanced.1', 'Advanced.2', 'Advanced.3', 'Year']
2003

In [14]:
df.columns = df.columns.str.strip()       # remove spaces
df.columns = df.columns.str.replace(r'\W+', '_', regex=True)  # clean non-alphanumerics
df.columns = df.columns.str.upper()       # normalize casing
rename_map = {
    "PK": "PK",
    "PICK": "PK",
    "ROUND_1_PK": "PK",
    "RK_PK": "PK",
    "P_K": "PK"
}

df = df.rename(columns={c: "PK" for c in df.columns if c in rename_map})


In [19]:
def flatten_columns(df):
    """
    Properly flatten Basketball-Reference multi-level column headers.
    Removes Unnamed levels and joins real header parts.
    """
    new_cols = []
    for col in df.columns:
        # col is a tuple for MultiIndex (level0, level1, ...)
        parts = [str(c) for c in col if "Unnamed" not in str(c)]
        if len(parts) == 0:
            parts = [""]  # empty fallback
        new_cols.append("_".join(parts).strip("_"))
    df.columns = new_cols
    return df

df = flatten_columns(df)


In [21]:
import pandas as pd
import requests
import time
import os

BASE_URL = "https://www.basketball-reference.com/draft/NBA_{}.html"
OUT_DIR = "draft_tables_fixed"
os.makedirs(OUT_DIR, exist_ok=True)

headers = {
    "User-Agent": "Mozilla/5.0"
}

def flatten_columns(df):
    """Properly flatten Basketball-Reference multi-level headers."""
    new_cols = []
    for col in df.columns:
        parts = [str(c) for c in col if ("Unnamed" not in str(c))]
        if len(parts) == 0:
            new_cols.append("")
        else:
            new_cols.append("_".join(parts))
    df.columns = new_cols
    return df

def scrape_year(year):
    print(f"Scraping {year}...")
    url = BASE_URL.format(year)
    html = requests.get(url, headers=headers).text

    tables = pd.read_html(html)
    df = tables[0].copy()

    # Flatten headers
    df = flatten_columns(df)

    # Drop empty columns
    df = df.loc[:, df.columns.notnull()]

    # Drop duplicate header rows inside table
    if "Pk" in df.columns:
        df = df[df["Pk"] != "Pk"]

    # Convert Pk to numeric
    if "Pk" in df.columns:
        df["Pk"] = pd.to_numeric(df["Pk"], errors="coerce")

    df["Year"] = year
    df.to_csv(f"{OUT_DIR}/nba_draft_{year}.csv", index=False)
    print(f"Saved nba_draft_{year}.csv")

for year in range(2000, 2026):
    scrape_year(year)
    time.sleep(1)


Scraping 2000...
Saved nba_draft_2000.csv


/tmp/ipython-input-1438856590.py:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)


Scraping 2001...
Saved nba_draft_2001.csv


/tmp/ipython-input-1438856590.py:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)


Scraping 2002...
Saved nba_draft_2002.csv


...

/tmp/ipython-input-1438856590.py:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)


Scraping 2024...
Saved nba_draft_2024.csv


/tmp/ipython-input-1438856590.py:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)


Scraping 2025...
Saved nba_draft_2025.csv


/tmp/ipython-input-1438856590.py:31: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)


In [23]:
import os

filename = "nba_draft_2000.csv"
for root, dirs, files in os.walk("/", topdown=True):
    if filename in files:
        print(os.path.join(root, filename))
        break



/content/draft_tables_fixed/nba_draft_2000.csv


In [24]:
import pandas as pd
import os

folder = "draft_tables_fixed"
dfs = []

for year in range(2000, 2026):
    df = pd.read_csv(os.path.join(folder, f"nba_draft_{year}.csv"))
    dfs.append(df)

df_all = pd.concat(dfs, ignore_index=True)

# Sorting now works because 'Pk' finally exists
df_all = df_all.sort_values(["Year", "Pk"]).reset_index(drop=True)

df_all.to_csv("nba_draft_2000_2025_all.csv", index=False)

df_all.head()


,Rk,Pk,Tm,Round 1_Player,Round 1_College,Yrs,Totals_G,Totals_MP,Totals_PTS,Totals_TRB,...,Shooting_FT%,Per Game_MP,Per Game_PTS,Per Game_TRB,Per Game_AST,Advanced_WS,Advanced_WS/48,Advanced_BPM,Advanced_VORP,Year
0,1.0,1.0,NJN,Kenyon Martin,Cincinnati,15.0,757,23134,9325,5159,...,.629,30.6,12.3,6.8,1.9,48.0,.100,0.1,12.1,2000
1,2.0,2.0,VAN,Stromile Swift,LSU,9.0,547,10804,4582,2535,...,.699,19.8,8.4,4.6,0.5,21.3,.095,-1.6,1.1,2000
2,3.0,3.0,LAC,Darius Miles,NaN,7.0,446,11730,4507,2190,...,.590,26.3,10.1,4.9,1.9,9.5,.039,-1.0,3.0,2000
3,4.0,4.0,CHI,Marcus Fizer,Iowa State,6.0,289,6032,2782,1340,...,.691,20.9,9.6,4.6,1.2,2.7,.022,-3.7,-2.6,2000
4,5.0,5.0,ORL,Mike Miller,Florida,17.0,1032,27812,10973,4376,...,.769,26.9,10.6,4.2,2.6,60.7,.105,0.8,19.8,2000


In [25]:
def strong_flatten(df):
    new_cols = []
    for col in df.columns:
        parts = [str(c) for c in col.split("_") if "Unnamed" not in c and c != ""]
        if len(parts) == 0:
            new_cols.append("COL")
        else:
            new_cols.append("_".join(parts))
    df.columns = new_cols
    return df


In [26]:
def clean_columns(df):
    clean = []
    for c in df.columns:
        c = str(c)
        c = c.replace(" ", "_")          # replace spaces
        c = c.replace("__", "_")         # collapse double underscores
        c = c.replace("Round_1_", "")    # remove extra hierarchy
        c = c.replace("Totals_", "")     # keep G, MP, PTS, etc.
        c = c.replace("Shooting_", "")   # Shooting FT% → FT%
        c = c.replace("Per_Game_", "")   # Per Game MP → MP (per-game)
        c = c.replace("Advanced_", "")   # remove advanced prefix
        clean.append(c.strip("_"))
    df.columns = clean
    return df

df_all = clean_columns(df_all)
df_all.head()


,Rk,Pk,Tm,Player,College,Yrs,G,MP,PTS,TRB,...,FT%,MP,PTS,TRB,AST,WS,WS/48,BPM,VORP,Year
0,1.0,1.0,NJN,Kenyon Martin,Cincinnati,15.0,757,23134,9325,5159,...,.629,30.6,12.3,6.8,1.9,48.0,.100,0.1,12.1,2000
1,2.0,2.0,VAN,Stromile Swift,LSU,9.0,547,10804,4582,2535,...,.699,19.8,8.4,4.6,0.5,21.3,.095,-1.6,1.1,2000
2,3.0,3.0,LAC,Darius Miles,NaN,7.0,446,11730,4507,2190,...,.590,26.3,10.1,4.9,1.9,9.5,.039,-1.0,3.0,2000
3,4.0,4.0,CHI,Marcus Fizer,Iowa State,6.0,289,6032,2782,1340,...,.691,20.9,9.6,4.6,1.2,2.7,.022,-3.7,-2.6,2000
4,5.0,5.0,ORL,Mike Miller,Florida,17.0,1032,27812,10973,4376,...,.769,26.9,10.6,4.2,2.6,60.7,.105,0.8,19.8,2000


In [27]:
# Ensure Year is numeric
df_all["Year"] = pd.to_numeric(df_all["Year"], errors="coerce").astype(int)

# Sort by Year then pick
df_all = df_all.sort_values(["Year", "Pk"]).reset_index(drop=True)

# Save final dataset
df_all.to_csv("nba_draft_2000_2025_clean.csv", index=False)

# Quick sanity checks
print(df_all[["Year", "Pk", "Tm", "Player"]].head(10))   # first 10 picks of 2000
print()
print(df_all[df_all["Year"] == 2001][["Year", "Pk", "Tm", "Player"]].head())  # start of 2001
print()
print(df_all["Year"].min(), df_all["Year"].max(), len(df_all))


   Year    Pk   Tm           Player
0  2000   1.0  NJN    Kenyon Martin
1  2000   2.0  VAN   Stromile Swift
2  2000   3.0  LAC     Darius Miles
3  2000   4.0  CHI     Marcus Fizer
4  2000   5.0  ORL      Mike Miller
5  2000   6.0  ATL  DerMarr Johnson
6  2000   7.0  CHI       Chris Mihm
7  2000   8.0  CLE   Jamal Crawford
8  2000   9.0  HOU   Joel Przybilla
9  2000  10.0  ORL    Keyon Dooling

    Year   Pk   Tm            Player
59  2001  1.0  WAS       Kwame Brown
60  2001  2.0  LAC    Tyson Chandler
61  2001  3.0  ATL         Pau Gasol
62  2001  4.0  CHI        Eddy Curry
63  2001  5.0  GSW  Jason Richardson

2000 2025 1578


In [28]:
from google.colab import files
files.download("nba_draft_2000_2025_clean.csv")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>